In [98]:
import os
import random
import time
from datetime import datetime
import csv
import pandas as pd
from sklearn.linear_model import LinearRegression

# Create output directory
output_dir = "sensor_data/csv"
os.makedirs(output_dir, exist_ok=True)


In [99]:
def classify_health_environment(voc, nh3, h2s, no2):
    score = 0
      # NH₃ (Ammoniak) – z.b Uringeruch
    if nh3 is not None and nh3 > 5:       #5
        score += 1
    # H₂S (Schwefelwasserstoff) – stinkendes Gas
    if h2s is not None and h2s > 0.005:   #0.005
        score += 1
    # VOC (flüchtige organische Verbindungen) – allgemeine Geruchsquelle
    if voc is not None and voc > 450:     #450
        score += 1
    # NO₂ (Stickstoffdioxid) – Schlechte Belüftung
    if no2 is not None and no2 > 300:     #300 Kommt nie vor, no2 hat kein geruch 250 vorher
        score += 1

    if score == 0:
        return "Fresh"
    elif score == 1:
        return "Noticeable"
    else:
        return "Stinky"

In [100]:
# Function to generate random air quality data
def generate_air_quality_data(sensor_id=3):
    nh3 = round(random.uniform(0, 10), 2)
    h2s = round(random.uniform(0, 0.01), 4)
    voc = round(random.uniform(50, 600), 2)
    no2 = round(random.uniform(0, 300), 2) if random.random() > 0.2 else None   #damit ein Wert mal fehlt 

    health_environment = classify_health_environment(voc, nh3, h2s, no2)

    return {
        "sensor_id": sensor_id,
        "timestamp": datetime.now().isoformat(),
        "nh3": nh3,
        "h2s": h2s,
        "voc": voc,
        "no2": no2,
        "health_environment": health_environment.lower()
    }

In [101]:
# Impute only no2 values using mean, median, mode or regression
def impute_missing_values(df, strategy='mean'):
    if strategy == 'mean':
        df['no2'] = df['no2'].fillna(df['no2'].mean()).round(2)
    elif strategy == 'median':
         df['no2'] = df['no2'].fillna(df['no2'].median()).round(2)
    elif strategy == 'mode':
        df['no2'] = df['no2'].fillna(df['no2'].mode()).round(2)
    #elif strategy == 'regression':             Haben keinen direkten Zusammenhang zwischen den Werten weshalb des net funktioniert
        # Only impute no2 values using so2 as predictor
    #    hum_train = df.dropna(subset=['no2'])
    #    model = LinearRegression()
    #    model.fit(hum_train[['so2']], hum_train['no2'])
    #    hum_missing = df[df['no2'].isnull()]
    #    df.loc[df['no2'].isnull(), 'no2'] = model.predict(hum_missing[['so2']]).round(2)
    return df

Daten in CSV Speichern

In [102]:
# Function to save data as CSV
def save_air_quality_csv(filename, data):
    if not data:
        return
    filepath = os.path.join(output_dir, filename)
    with open(filepath, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"Saved Air Quality CSV: {filepath}")

# User input for number of times to run
num_iterations = int(input("Enter number of air quality readings to generate: "))

# Generate multiple batches of air quality data
readings = []
for i in range(num_iterations):
    air_quality_data = generate_air_quality_data()
    print(air_quality_data)
    readings.append(air_quality_data)
    time.sleep(0.5)  # Simulate real-time data generation

    # Add a duplicate at every 3rd reading
    if i % 3 == 0 and i != 0:
        readings.append(air_quality_data)
        print(air_quality_data)

# Create DataFrame
df = pd.DataFrame(readings)

# Save raw data (with duplicates)
save_air_quality_csv("air_quality_readings_errors.csv", df.to_dict(orient='records'))

{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:21.322128', 'nh3': 6.76, 'h2s': 0.0068, 'voc': 392.5, 'no2': None, 'health_environment': 'stinky'}
{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:21.823476', 'nh3': 0.87, 'h2s': 0.0073, 'voc': 227.45, 'no2': 193.68, 'health_environment': 'noticeable'}
{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:22.324760', 'nh3': 6.63, 'h2s': 0.0076, 'voc': 396.8, 'no2': 241.98, 'health_environment': 'stinky'}
{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:22.826277', 'nh3': 9.78, 'h2s': 0.0088, 'voc': 195.47, 'no2': 275.68, 'health_environment': 'stinky'}
{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:22.826277', 'nh3': 9.78, 'h2s': 0.0088, 'voc': 195.47, 'no2': 275.68, 'health_environment': 'stinky'}
{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:23.329737', 'nh3': 0.35, 'h2s': 0.0044, 'voc': 252.93, 'no2': 12.86, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T14:38:23.830378', 'nh3': 0.78, 'h2s': 0.0012, 'voc': 72.32, 

Doppelte Werte entfernen und Fehlende Ergänzen:

In [103]:
# Remove duplicated rows
df = df.drop_duplicates()

# Impute missing no2 values 
df = impute_missing_values(df, strategy="mean")  #besser, aber anfälliger für ausreißer

# Save cleaned data (without duplicates)
save_air_quality_csv("air_quality_readings.csv", df.to_dict(orient='records'))

#df.to_csv("sensor_data/csv/air_quality_reading.csv", index=False)

Saved Air Quality CSV: sensor_data/csv\air_quality_readings.csv
